# 🎯 베이스라인 구현 - KoBART 기반 대화 요약
> PRD 계획에 따른 베이스라인 모델 구현

**목표 성능**: ROUGE-F1 47+

In [1]:
# 환경 설정
import sys
import os
from pathlib import Path

# 프로젝트 루트 경로 추가
notebook_dir = Path.cwd()
# notebooks/team/CHH -> notebooks -> team -> natural-language-processing-competition
project_root = notebook_dir.parent.parent.parent  # 3번만 parent 사용!

# 다른 프로젝트 경로 제거하고 현재 프로젝트 경로만 추가
sys.path = [p for p in sys.path if 'computer-vision-competition' not in p]
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project Root: {project_root}")
print(f"Current Dir: {notebook_dir}")

# 필요한 라이브러리 임포트
import yaml
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from transformers import AutoTokenizer, BartForConditionalGeneration
from rouge import Rouge
import wandb

# 커스텀 모듈 임포트
from src.logging.notebook_logger import NotebookLogger
from src.utils.gpu_optimization.team_gpu_check import check_gpu_tier

print("Libraries imported successfully!")

Project Root: /home/ieyeppo/AI_Lab/natural-language-processing-competition
Current Dir: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH
Libraries imported successfully!


In [2]:
# 설정 로드
config_path = notebook_dir / 'configs' / 'config_baseline.yaml'
with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print(f"Model: {config['model']['name']}")
print(f"Batch Size: {config['training']['batch_size']}")

Model: digit82/kobart-summarization
Batch Size: 16


In [ ]:
# 로거 초기화
# config의 로그 경로 사용
def get_path(path_str):
    """config의 상대 경로를 절대 경로로 변환"""
    path = Path(path_str)
    if not path.is_absolute():
        path = notebook_dir / path
    return path

log_dir = get_path(config['paths']['log_dir'])
log_dir.mkdir(parents=True, exist_ok=True)

# 타임스탬프 생성
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 로거 초기화
log_file = log_dir / f'baseline_{timestamp}.log'
logger = NotebookLogger(
    log_path=str(log_file),
    print_also=True
)

logger.write('=== Baseline Experiment Started ===')

=== Baseline Experiment Started ===


In [4]:
# GPU 체크
if torch.cuda.is_available():
    gpu_tier = check_gpu_tier()
    logger.write(f"GPU Tier: {gpu_tier}")
    logger.write(f"GPU: {torch.cuda.get_device_name(0)}")

GPU Tier: LOW
GPU: NVIDIA GeForce RTX 4090


In [5]:
# 데이터 경로 설정 및 로드
# config 파일의 경로 사용
def get_data_path(path_str):
    """config의 상대 경로를 절대 경로로 변환"""
    path = Path(path_str)
    if not path.is_absolute():
        path = notebook_dir / path
    return path

# config에서 데이터 경로 가져오기
train_path = get_data_path(config['paths']['train_file'])
dev_path = get_data_path(config['paths']['dev_file'])
test_path = get_data_path(config['paths']['test_file'])

logger.write(f"Loading data from config paths:")
logger.write(f"  - Train: {train_path}")
logger.write(f"  - Dev: {dev_path}")
logger.write(f"  - Test: {test_path}")

# 데이터 로드
train_df = pd.read_csv(train_path)
dev_df = pd.read_csv(dev_path)
test_df = pd.read_csv(test_path)

logger.write(f"Data loaded successfully!")
logger.write(f"Train samples: {len(train_df)}")
logger.write(f"Dev samples: {len(dev_df)}")
logger.write(f"Test samples: {len(test_df)}")

# 데이터 샘플 출력
print("\nSample data:")
print(train_df.head(2))

Loading data from config paths:
  - Train: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/train.csv
  - Dev: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/dev.csv
  - Test: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/test.csv
Data loaded successfully!
Train samples: 12457
Dev samples: 499
Test samples: 499

Sample data:
     fname                                           dialogue  \
0  train_0  #Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다...   
1  train_1  #Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?\n#Pers...   

                                             summary  topic  
0  Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필...   건강검진  
1  Mrs. Parker가 Ricky와 함께 백신 접종을 위해 방문하였고, Dr. Pe...  백신 접종  


In [6]:
# WandB 초기화
wandb.init(
    project=config['wandb']['project'],
    entity=config['wandb']['entity'],
    name=config['wandb']['name'],
    config=config
)

wandb: Currently logged in as: ieyeppo-job (kimsunmin0227-hufs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [7]:
# 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(config['model']['name'])
model = BartForConditionalGeneration.from_pretrained(config['model']['name'])

logger.write(f"Model loaded: {config['model']['name']}")

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


Model loaded: digit82/kobart-summarization


## 데이터 전처리 및 데이터셋 클래스 정의

In [8]:
# 데이터 전처리 함수
def preprocess_dialogue(text):
    """대화 텍스트 전처리"""
    # 노이즈 제거
    text = text.replace('\\n', '\n')
    text = text.replace('<br>', '\n')
    
    # 특수문자 정규화
    text = text.strip()
    
    # #Person 태그 최적화 (더 명확하게)
    import re
    text = re.sub(r'#Person(\d+)#:', r'화자\1:', text)
    
    return text

def preprocess_summary(text):
    """요약 텍스트 전처리"""
    if pd.isna(text):
        return ""
    text = text.strip()
    return text

# 데이터 전처리 적용
train_df['dialogue_preprocessed'] = train_df['dialogue'].apply(preprocess_dialogue)
train_df['summary_preprocessed'] = train_df['summary'].apply(preprocess_summary)

dev_df['dialogue_preprocessed'] = dev_df['dialogue'].apply(preprocess_dialogue)
dev_df['summary_preprocessed'] = dev_df['summary'].apply(preprocess_summary)

test_df['dialogue_preprocessed'] = test_df['dialogue'].apply(preprocess_dialogue)

print(f"전처리 완료!")
print(f"Sample preprocessed dialogue (first 200 chars):")
print(train_df['dialogue_preprocessed'].iloc[0][:200])
logger.write("Data preprocessing completed")

전처리 완료!
Sample preprocessed dialogue (first 200 chars):
화자1: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? 
화자2: 건강검진을 받으려고 왔어요. 
화자1: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. 
화자2: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요. 
화자1: 음, 심각한 질병을 피하려면 미리 발견하는 게 
Data preprocessing completed


In [9]:
# PyTorch Dataset 클래스 정의
from torch.utils.data import Dataset, DataLoader

class DialogueSummaryDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_input_len=512, max_target_len=128, is_test=False):
        """
        대화 요약 데이터셋
        
        Args:
            dataframe: 데이터프레임
            tokenizer: 토크나이저
            max_input_len: 최대 입력 길이
            max_target_len: 최대 타겟 길이
            is_test: 테스트 모드 여부
        """
        self.df = dataframe
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len
        self.is_test = is_test
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # 입력 텍스트
        dialogue = row['dialogue_preprocessed']
        
        # 입력 토큰화
        inputs = self.tokenizer(
            dialogue,
            max_length=self.max_input_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # 테스트 모드가 아닌 경우 타겟도 처리
        if not self.is_test:
            summary = row['summary_preprocessed']
            
            # 타겟 토큰화
            targets = self.tokenizer(
                summary,
                max_length=self.max_target_len,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            
            return {
                'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze(),
                'labels': targets['input_ids'].squeeze()
            }
        else:
            return {
                'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze(),
                'idx': idx
            }

# 데이터셋 생성
train_dataset = DialogueSummaryDataset(
    train_df, 
    tokenizer, 
    max_input_len=config['model']['max_input_length'],
    max_target_len=config['model']['max_target_length']
)

val_dataset = DialogueSummaryDataset(
    dev_df,
    tokenizer,
    max_input_len=config['model']['max_input_length'],
    max_target_len=config['model']['max_target_length']
)

test_dataset = DialogueSummaryDataset(
    test_df,
    tokenizer,
    max_input_len=config['model']['max_input_length'],
    max_target_len=config['model']['max_target_length'],
    is_test=True
)

logger.write(f"Dataset created - Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")
print(f"Dataset shapes:")
print(f"  Train: {len(train_dataset)}")
print(f"  Val: {len(val_dataset)}")
print(f"  Test: {len(test_dataset)}")

Dataset created - Train: 12457, Val: 499, Test: 499
Dataset shapes:
  Train: 12457
  Val: 499
  Test: 499


In [10]:
# DataLoader 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=config['training']['batch_size'],
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config['training']['batch_size'],
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config['training']['batch_size'],
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"DataLoader created:")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

DataLoader created:
  Train batches: 779
  Val batches: 32
  Test batches: 32


## 학습 및 평가 함수 정의

In [11]:
import torch.nn as nn
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm
import gc

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
logger.write(f"Using device: {device}")

# ROUGE 평가 함수
def compute_rouge_scores(predictions, references):
    """ROUGE 점수 계산"""
    rouge = Rouge()
    
    # 빈 문자열 처리
    predictions = [p if p else "empty" for p in predictions]
    references = [r if r else "empty" for r in references]
    
    try:
        scores = rouge.get_scores(predictions, references, avg=True)
        return {
            'rouge-1': scores['rouge-1']['f'],
            'rouge-2': scores['rouge-2']['f'],
            'rouge-l': scores['rouge-l']['f']
        }
    except Exception as e:
        logger.write(f"Error computing ROUGE: {e}")
        return {'rouge-1': 0, 'rouge-2': 0, 'rouge-l': 0}

# 학습 함수
def train_epoch(model, data_loader, optimizer, scheduler, device):
    """한 에폭 학습"""
    model.train()
    total_loss = 0
    
    progress_bar = tqdm(data_loader, desc='Training')
    
    for batch in progress_bar:
        # 데이터를 디바이스로 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # 그래디언트 초기화
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        total_loss += loss.item()
        
        # Backward pass
        loss.backward()
        
        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # 옵티마이저 스텝
        optimizer.step()
        scheduler.step()
        
        # 프로그레스 바 업데이트
        progress_bar.set_postfix({'loss': loss.item()})
        
        # WandB 로깅
        wandb.log({
            'train_loss': loss.item(),
            'learning_rate': scheduler.get_last_lr()[0]
        })
    
    avg_loss = total_loss / len(data_loader)
    return avg_loss

# 검증 함수
def evaluate(model, data_loader, tokenizer, device, num_samples=None):
    """모델 평가"""
    model.eval()
    total_loss = 0
    predictions = []
    references = []
    
    with torch.no_grad():
        progress_bar = tqdm(data_loader, desc='Evaluating')
        
        for i, batch in enumerate(progress_bar):
            if num_samples and i >= num_samples:
                break
                
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Loss 계산
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            total_loss += outputs.loss.item()
            
            # 예측 생성 - config 키 수정
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=config['model']['max_target_length'],
                num_beams=config['evaluation']['num_beams'],  # config 키 수정
                early_stopping=True,
                no_repeat_ngram_size=config['evaluation']['no_repeat_ngram_size']
            )
            
            # 디코딩
            preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            refs = tokenizer.batch_decode(labels, skip_special_tokens=True)
            
            predictions.extend(preds)
            references.extend(refs)
    
    # ROUGE 점수 계산
    rouge_scores = compute_rouge_scores(predictions, references)
    avg_loss = total_loss / len(data_loader)
    
    return avg_loss, rouge_scores, predictions[:5]  # 샘플 예측 반환

print("Training functions defined successfully!")

Using device: cuda
Training functions defined successfully!


In [12]:
# 옵티마이저 및 스케줄러 설정
# config 값들을 안전하게 가져오기
num_epochs = config['training'].get('num_epochs', config['training'].get('epochs', 3))
learning_rate = config['training']['learning_rate']
# learning_rate가 문자열인 경우 float로 변환
if isinstance(learning_rate, str):
    learning_rate = float(learning_rate)
    print(f"Learning rate converted from string to float: {learning_rate}")

num_training_steps = num_epochs * len(train_loader)

optimizer = AdamW(
    model.parameters(),
    lr=learning_rate,  # 이미 float로 변환됨
    weight_decay=config['training']['weight_decay']
)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(num_training_steps * config['training']['warmup_ratio']),
    num_training_steps=num_training_steps
)

logger.write(f"Optimizer and scheduler initialized")
logger.write(f"Learning rate: {learning_rate}")
logger.write(f"Total training steps: {num_training_steps}")
logger.write(f"Warmup steps: {int(num_training_steps * config['training']['warmup_ratio'])}")

# 학습 기록 저장
training_history = {
    'train_loss': [],
    'val_loss': [],
    'rouge_1': [],
    'rouge_2': [],
    'rouge_l': []
}

# Early Stopping 설정
best_rouge_l = 0
patience = config['training']['early_stopping_patience']
patience_counter = 0

# 모델 저장 경로 - config의 경로 사용
model_dir = get_path(config['paths']['output_dir'])
model_dir.mkdir(parents=True, exist_ok=True)
best_model_path = model_dir / 'best_model.pt'

logger.write("=" * 50)
logger.write("Starting training...")
logger.write("=" * 50)

# 학습 루프
for epoch in range(num_epochs):
    logger.write(f"\nEpoch {epoch + 1}/{num_epochs}")
    logger.write("-" * 30)
    
    # 학습
    train_loss = train_epoch(model, train_loader, optimizer, scheduler, device)
    logger.write(f"Average training loss: {train_loss:.4f}")
    training_history['train_loss'].append(train_loss)
    
    # 검증
    val_loss, rouge_scores, sample_preds = evaluate(model, val_loader, tokenizer, device)
    
    logger.write(f"Validation loss: {val_loss:.4f}")
    logger.write(f"ROUGE-1 F1: {rouge_scores['rouge-1']:.4f}")
    logger.write(f"ROUGE-2 F1: {rouge_scores['rouge-2']:.4f}")
    logger.write(f"ROUGE-L F1: {rouge_scores['rouge-l']:.4f}")
    
    # 학습 기록 저장
    training_history['val_loss'].append(val_loss)
    training_history['rouge_1'].append(rouge_scores['rouge-1'])
    training_history['rouge_2'].append(rouge_scores['rouge-2'])
    training_history['rouge_l'].append(rouge_scores['rouge-l'])
    
    # WandB 로깅
    wandb.log({
        'epoch': epoch + 1,
        'train_loss_epoch': train_loss,
        'val_loss': val_loss,
        'rouge_1': rouge_scores['rouge-1'],
        'rouge_2': rouge_scores['rouge-2'],
        'rouge_l': rouge_scores['rouge-l']
    })
    
    # 샘플 예측 출력
    logger.write("\nSample predictions:")
    for i, pred in enumerate(sample_preds[:2]):
        logger.write(f"  Sample {i+1}: {pred[:100]}...")
    
    # Best model 저장
    if rouge_scores['rouge-l'] > best_rouge_l:
        best_rouge_l = rouge_scores['rouge-l']
        patience_counter = 0
        
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'rouge_scores': rouge_scores,
            'config': config
        }, best_model_path)
        
        logger.write(f"✓ New best model saved! (ROUGE-L: {best_rouge_l:.4f})")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            logger.write(f"Early stopping triggered after {epoch + 1} epochs")
            break
    
    # 메모리 정리
    torch.cuda.empty_cache()
    gc.collect()

logger.write("\n" + "=" * 50)
logger.write(f"Training completed!")
logger.write(f"Best ROUGE-L: {best_rouge_l:.4f}")
logger.write("=" * 50)

Learning rate converted from string to float: 5e-05
Optimizer and scheduler initialized
Learning rate: 5e-05
Total training steps: 7790
Warmup steps: 779
Starting training...

Epoch 1/10
------------------------------


Training:   0%|          | 0/779 [00:00<?, ?it/s]

Average training loss: 1.5061


Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

Validation loss: 0.5410
ROUGE-1 F1: 0.1548
ROUGE-2 F1: 0.0446
ROUGE-L F1: 0.1490

Sample predictions:
  Sample 1: #Person1#은 감기에 걸렸다고 말하며, 천식 검사를 위해 폐 전문의에게 가보라고 제안합니다.
세요?
어.
!
㗡까요?
...
  Sample 2: #Person1#은 Jimmy에게 운동하러 가자고 제안하고, Jammi는 토요일에 운동하자고 제안한다. 그들은 오후 3시 30분에 체육관에서 만나기로 한다. 그들은 체육관 운동에 ...
✓ New best model saved! (ROUGE-L: 0.1490)

Epoch 2/10
------------------------------


...

Validation loss: 0.5182
ROUGE-1 F1: 0.1382
ROUGE-2 F1: 0.0389
ROUGE-L F1: 0.1315

Sample predictions:
  Sample 1: #Person2#는 최근 숨쉬기 어려움을 겪고 있으며, 천식 검사를 위해 폐 전문의에게 방문할 것을 권장하고 있습니다. 의사는 천식을 검사하기 위해 의사를 방문해야 한다고 안내합니...
  Sample 2: #Person1#이 Jimmy에게 운동을 하자고 제안하지만, 그녀는 다리가 아파서 거절한다. #My는 주간 일정을 변경하자고 제안하고, Jammi는 이를 수락한다. 결국 그들은 3...

Epoch 4/10
------------------------------


Training:   0%|          | 0/779 [00:00<?, ?it/s]

Average training loss: 0.3052


Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

Validation loss: 0.5443
ROUGE-1 F1: 0.1215
ROUGE-2 F1: 0.0329
ROUGE-L F1: 0.1162

Sample predictions:
  Sample 1: #Person2#는 최근 숨쉬기 어려움을 겪고 있으며, 천식 검사를 위해 폐 전문의에게 가볼 것을 권장합니다.세요?
어요?
어.
까요?
 
!
어요.
겠습니다.
예요.
.
더라.
...
  Sample 2: #Person1#이 Jimmy에게 운동 일정에 대해 물어보고, 결국 그들은 3시 30분에 체육관에서 함께 운동하기로 결정한다.세요?
어요?
 
까요?
겠습니다.
어.
어요.
!
....
Early stopping triggered after 4 epochs

Training completed!
Best ROUGE-L: 0.1490


## 테스트 데이터 예측 및 제출 파일 생성

In [13]:
# 최적 모델 로드
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model_state_dict'])
logger.write(f"Best model loaded from epoch {checkpoint['epoch'] + 1}")
logger.write(f"Best ROUGE scores: {checkpoint['rouge_scores']}")

# 테스트 데이터 예측
def generate_predictions(model, data_loader, tokenizer, device):
    """테스트 데이터에 대한 예측 생성"""
    model.eval()
    all_predictions = []
    all_indices = []
    
    with torch.no_grad():
        progress_bar = tqdm(data_loader, desc='Generating predictions')
        
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            indices = batch['idx']
            
            # 예측 생성
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=config['inference']['max_length'],
                num_beams=config['inference']['num_beams'],
                early_stopping=config['inference']['early_stopping'],
                no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                length_penalty=config['inference']['length_penalty'],
                temperature=config['inference']['temperature']
            )
            
            # 디코딩
            predictions = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            
            all_predictions.extend(predictions)
            all_indices.extend(indices.tolist())
    
    # 인덱스 순서대로 정렬
    sorted_predictions = [pred for _, pred in sorted(zip(all_indices, all_predictions))]
    
    return sorted_predictions

# 예측 수행
logger.write("\nGenerating predictions for test set...")
test_predictions = generate_predictions(model, test_loader, tokenizer, device)
logger.write(f"Generated {len(test_predictions)} predictions")

# 샘플 출력
print("\nSample test predictions:")
for i in range(min(3, len(test_predictions))):
    print(f"Test {i+1}: {test_predictions[i][:150]}...")
    print("-" * 50)

Best model loaded from epoch 1
Best ROUGE scores: {'rouge-1': 0.154792565273082, 'rouge-2': 0.04464649129948705, 'rouge-l': 0.1490111431801561}

Generating predictions for test set...


Generating predictions:   0%|          | 0/32 [00:00<?, ?it/s]

Generated 499 predictions

Sample test predictions:
Test 1: #Person1#은 Ms. Dawson에게 사내 메모를 작성하고 배포해달라고 요청합니다. Mrsy는 메시지를 계속 사용하면 경고 후 시정 조치가 내려질 수 있다고 설명합니다.
어.
세요?
!
㗡까요?
...
--------------------------------------------------
Test 2: #Person1#은 출퇴근 시간에 교통체증으로 인해 교통 체증 때문에 대중교통을 이용해야 한다고 말합니다. 그들은 환경에도 도움이 될 것이라고 생각합니다.어.

세요?
㗡!
까요?
...
--------------------------------------------------
Test 3: Kate는 #Person1#에게 Masha와 Hero가 두 달 동안 별거 중이며 이혼을 했다고 말합니다. Kata는 이를 믿기 어렵다고 생각하지만, Misha가 양육권을 가지기로 했다고 전합니다.
세요?
어.
㗡!
까요?
...
--------------------------------------------------


In [16]:
# 제출 파일 생성
submission_df = pd.DataFrame({
    'fname': test_df['fname'],
    'summary': test_predictions
})

# 제출 파일 저장 - config의 경로 사용
submission_dir = get_path(config['paths']['submission_dir'])
submission_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
submission_filename = f'baseline_submission_{timestamp}.csv'
submission_path = submission_dir / submission_filename

# index=True로 설정하여 인덱스를 포함시킴
submission_df.to_csv(submission_path, index=True, encoding='utf-8')  # index=False -> index=True로 변경
logger.write(f"\nSubmission file saved: {submission_path}")

# 제출 파일 확인
print(f"\nSubmission file created: {submission_filename}")
print(f"Shape: {submission_df.shape}")
print("\nFirst 3 submissions:")
print(submission_df.head(3))

# 최종 요약 통계
print("\n" + "=" * 50)
print("BASELINE EXPERIMENT SUMMARY")
print("=" * 50)
print(f"Model: {config['model']['name']}")
print(f"Best ROUGE-L: {best_rouge_l:.4f}")
print(f"Training epochs: {len(training_history['train_loss'])}")
print(f"Final train loss: {training_history['train_loss'][-1]:.4f}")
print(f"Final val loss: {training_history['val_loss'][-1]:.4f}")
print(f"Submission file: {submission_filename}")
print("=" * 50)

# WandB 실험 종료
wandb.finish()

logger.write("\n✅ Baseline experiment completed successfully!")
logger.write(f"Log file: {log_dir / 'baseline.log'}")


Submission file saved: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/submissions/baseline/baseline_submission_20251010_101823.csv

Submission file created: baseline_submission_20251010_101823.csv
Shape: (499, 2)

First 3 submissions:
    fname                                            summary
0  test_0  #Person1#은 Ms. Dawson에게 사내 메모를 작성하고 배포해달라고 요청합...
1  test_1  #Person1#은 출퇴근 시간에 교통체증으로 인해 교통 체증 때문에 대중교통을 이...
2  test_2  Kate는 #Person1#에게 Masha와 Hero가 두 달 동안 별거 중이며 이...

BASELINE EXPERIMENT SUMMARY
Model: digit82/kobart-summarization
Best ROUGE-L: 0.1490
Training epochs: 4
Final train loss: 0.3052
Final val loss: 0.5443
Submission file: baseline_submission_20251010_101823.csv

✅ Baseline experiment completed successfully!
Log file: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/logs/baseline/baseline.log
